In [20]:
import pandas as pd
import numpy as np
enronEmails = pd.read_pickle('email_scripts/enronEmails.pkl')

In [15]:
print(enronEmails['body'][8])

Nelson,  Here is the latest version of the MMS Keep Whole that I have.
I believe that we were waiting on the language from Gerald that tweeked their relevant gas day language.
Let me know if you need the latest version of the language that MMS provided to us with the Relevant Gas day language.
Thanks,



In [21]:
signoffs = ['Regards','Thanks','Best','Best Regards','Sincerely','Cheers','Yours','Cordially','Faithfully','Respectfully','Warmly','Kindly','Take care','Best wishes','All the best','Yours truly','Yours sincerely','Yours faithfully','Yours cordially','Yours respectfully','Yours warmest','Yours kindest','Yours most sincerely','Yours most cordially','Yours most respectfully','Yours most warmest','Yours most kindest']

#find emails with signoffs
emails_with_signoffs = []
train_indices = []
for idx, email in enumerate(enronEmails['body']):
    for signoff in signoffs:
        if signoff in email:
            emails_with_signoffs.append(email)
            train_indices.append(idx)
            # print(email)
            # break
print(len(emails_with_signoffs))

5368


In [34]:
blizzard_signature = """
Blizzard T. Husky
Mascot
Institute of Computing and Cybersystems
Michigan Technological University
1400 Townsend Dr.
Houghton, MI 49931
906.555.1234
blizzardThusky@mtu.edu
"""

#randomly sample 100 of the emails
import random
random.seed(0)
random_indices = random.sample(list(np.arange(len(emails_with_signoffs))), 100)

for index in random_indices:
    # append these emails with the signature
    text = emails_with_signoffs[index]
    #find the index of the signoff
    for signoff in signoffs:
        if signoff in text:
            signoff_index = text.find(signoff)
            break

    #replace the signoff with the signature
    emails_with_signoffs[index] = text[:signoff_index] + signoff + ',' + blizzard_signature
    


In [36]:
print(emails_with_signoffs[index])

Attached below are the two files for TW CMS/TMS requirements on flow direction and current process for inside/outside the path requirements.
Please review these documents and let me know if you have any questions or  corrections to apply to either document by COB 8/31/01.
Linda, please keep me in the loop on both of the projects since they interface directly with TMS.
Yours most kindest,
  Blizzard T. Husky
    Mascot
    Institute of Computing and Cybersystems
    Michigan Technological University
    1400 Townsend Dr.
    Houghton, MI 49931
    906.555.1234
    blizzardThusky@mtu.edu

